# Introduction to Agno & Architecture (Student Copy)

**Goal:** Understand Agno’s core concepts and system architecture so you can reason about how agents, teams, workflows, tools, memory, and the runtime (AgentOS) fit together.

## 1) High‑Level Overview

Agno = **multi‑agent framework** + **runtime (AgentOS)** + **control plane (web UI)**.

- Build **Agents**, combine them into **Teams** (autonomy), or orchestrate with **Workflows** (control).  
- Run everything on **AgentOS (FastAPI)**.  
- Use the **AgentOS UI** to test, monitor, and manage in real time.  
- **Private by design**: The runtime lives in your environment; the UI connects directly (no external data retention).


### Big‑Picture Architecture
```mermaid
flowchart LR
    subgraph Client
        U[User / App / CLI]
        UI[AgentOS UI Browser]
    end

    subgraph Runtime
        OS[AgentOS FastAPI]
        ST[(Persistent Storage<br/>DB / Files / Vector DB)]
    end

    subgraph AgentsLayer[Agents / Teams / Workflows]
        A1[Agent]
        T1[Team]
        W1[Workflow]
    end

    subgraph Ext[External Integrations]
        TOOLS[Tools / APIs / Services]
        MCP[(MCP Servers)]
        KNOW[(Knowledge / Vector Stores)]
    end

    U <-->|HTTP / SSE| OS
    UI <-->|WS / SSE| OS
    OS --> A1
    OS --> T1
    OS --> W1
    A1 <-->|state/history| ST
    T1 <-->|shared ctx| ST
    W1 <-->|session state| ST
    A1 -->|tool calls| TOOLS
    A1 -->|MCP| MCP
    A1 -->|RAG| KNOW
    
    style U fill:#e1f5fe
    style UI fill:#e1f5fe
    style OS fill:#f3e5f5
    style ST fill:#e3f2fd
    style A1 fill:#e8f5e8
    style T1 fill:#e8f5e8
    style W1 fill:#e8f5e8
```

## 2) Core Building Blocks

| Component | What it is | Why it matters |
|---|---|---|
| **Agent** | An LLM‑powered actor with instructions, tools, memory. | Encapsulates role, reasoning, and actions for a task. |
| **Team** | A group of agents collaborating with shared context. | Increases autonomy and specialization (researcher + analyst, etc.). |
| **Workflow** | A deterministic, step‑based orchestration. | Adds explicit control: branching, looping, parallel steps. |
| **AgentOS** | FastAPI runtime that serves agents/teams/workflows. | Production‑ready API, streaming endpoints, scaling. |
| **Control Plane (UI)** | Web UI connecting to your AgentOS. | Test, monitor, and debug runs, tools, and memory live. |
| **Tools** | Built‑in or custom functions/APIs the agent can call. | Extend capability beyond LLM knowledge (web, data, code). |
| **Memory** | User/session state persisted to storage. | Context across turns; personalization. |
| **Knowledge** | Vector‑store backed retrieval (RAG). | Ground answers in your docs/data. |
| **MCP** | Model Context Protocol integration. | Standard way to connect external systems securely. |
| **Hooks & Guardrails** | Pre/post validation & safety mechanisms. | Enforce structure, protect prompts, confirm actions. |


## 3) Orchestration Styles: Team vs Workflow

Use **Teams** when you want **agent autonomy and collaboration**.  
Use **Workflows** when you need **explicit sequences and routing**.

```mermaid
flowchart TB
    subgraph Team
        UQ[User Query] --> LEAD[Team Leader]
        LEAD --> R1[Agent A: Research]
        LEAD --> R2[Agent B: Analyze]
        LEAD --> R3[Agent C: Write]
        R1 -->|share| LEAD
        R2 -->|share| LEAD
        R3 -->|share| LEAD
        LEAD --> OUT[Final Answer]
    end
    
    style UQ fill:#e1f5fe
    style LEAD fill:#fff3e0
    style R1 fill:#e8f5e8
    style R2 fill:#e8f5e8
    style R3 fill:#e8f5e8
    style OUT fill:#e3f2fd
```
---



```mermaid
flowchart LR
  subgraph Workflow
    A[Step 1: Parse Intent] -->|branch| B{Router}
    B -->|code| C[Step 2: Coding Agent]
    B -->|general| D[Step 2: QA Agent]
    C --> E[Step 3: Tests]
    D --> E
    E --> F[Step 4: Format / Validate]
  end
```


### Quick Comparison

| Aspect | Team | Workflow |
|---|---|---|
| **Control** | Emergent, agent‑driven | Explicit, developer‑defined |
| **When to use** | Open‑ended tasks, collaboration | Deterministic pipelines, forms, routing |
| **Parallelism** | Natural via multiple agents | Explicit (mark steps parallel) |
| **Debuggability** | Use UI traces to see dialogue | Inspect step outputs and branches |
| **Maintenance** | Adjust roles/instructions | Adjust DAG (steps/conditions) |


## 4) Inside an Agent

**Key parts**  
- **Model**: any supported LLM provider (model‑agnostic).  
- **Instructions**: role/system prompt that sets behavior.  
- **Tools**: functions/APIs the agent may invoke.  
- **Structured I/O**: `input_schema` / `output_schema` for predictable JSON.  
- **Memory**: user/session history and state persisted to storage.  
- **Markdown / Multimodal**: format output, handle images/audio/files.

```mermaid
flowchart LR
  subgraph Agent
    I[Instructions] --> LLM[(Model)]
    SCH[Input / Output Schemas] --> LLM
    MEM[(Memory/State)] --> LLM
    LLM --> O[Answer]
    LLM --> TC{Tool Call?}
    TC --> TOOL[Tool]
    TOOL --> LLM
  end
```


### Structured I/O (type‑safe)

| Feature | Purpose | Example |
|---|---|---|
| **`input_schema`** | Validate/shape incoming params | Enforce required fields/types |
| **`output_schema`** | Force JSON output structure | `{title:str, rating:float, url:str}` |
| **Validation hooks** | Post‑process results | Reject/massage invalid values |


### Reference: Simple Agent with Structured Output (do not run without creds)

In [ ]:
from pydantic import BaseModel, Field
from agno.agent import Agent
from agno.models.openai import OpenAIChat  # provider example

class MovieSuggestion(BaseModel):
    title: str = Field(..., description="Movie title")
    genre: str
    rating: float

agent = Agent(
    name="Recommender",
    model=OpenAIChat(id="gpt-3.5-turbo"),
    output_schema=MovieSuggestion,
    instructions="Recommend one movie that fits the user's mood."
)

print(agent.run("I want a light-hearted comedy tonight."))

## 5) State, Memory & Persistence

- **Session:** connects turns in a conversation (via `session_id`).  
- **User memory:** store user‑specific facts across sessions (`user_id`).  
- **Persistent storage:** SQLite / files / other DBs for history & state.  
- **Knowledge (RAG):** connect 20+ vector stores to retrieve relevant docs.  
- **Culture:** shared, compounding knowledge across agents.

```mermaid
flowchart LR
  subgraph Persistence
    DB[(Relational/JSON)]
    VS[(Vector Store)]
  end
  AG[Agent] -->|save history| DB
  AG -->|retrieve chunks| VS
  T[Team] -->|shared ctx| DB
  W[Workflow] -->|session state| DB
```


## 6) AgentOS Runtime & Scaling

**AgentOS** is a **FastAPI** runtime with **async** endpoints and **SSE** streaming.

- **Stateless at runtime** → horizontally scalable behind a load balancer.  
- **Low overhead** → microseconds agent instantiation, tiny memory footprint.  
- **Private by design** → run entirely in your cloud; UI connects directly.

```mermaid
flowchart LR
  subgraph Cluster
    LB[Load Balancer]
    S1[AgentOS Instance 1]
    S2[AgentOS Instance 2]
    S3[AgentOS Instance 3]
  end
  LB --> S1
  LB --> S2
  LB --> S3
  S1 --- ST[(Shared Storage/DB)]
  S2 --- ST
  S3 --- ST
```


## 7) Tools & Integrations

- **Built‑in toolkits**: 100+ toolkits across web/data/code/enterprise.  
- **Custom tools**: any Python function (or `@tool` decorator).  
- **MCP**: first‑class support to connect external systems safely.  
- **Best practices**: clear docstrings; require confirmation for sensitive actions; add caching for expensive calls.


In [ ]:
# Reference only: add a simple custom tool and a built-in tool
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.duckduckgo import DuckDuckGoTools

def get_stock_price(symbol: str) -> str:
    # Replace with real API in production
    return f"{symbol} is trading at $123.45 (demo)"

finance_agent = Agent(
    name="FinanceHelper",
    model=OpenAIChat(id="gpt-3.5-turbo"),
    tools=[DuckDuckGoTools(), get_stock_price],
    instructions="Use web search and the stock tool when helpful; answer clearly."
)

print(finance_agent.run("What is the latest on NVIDIA and how is NVDA trading?"))

## 8) Security, Governance & Reliability

- **Guardrails:** validate outputs, protect prompts, enforce policies.  
- **Human‑in‑the‑Loop:** confirmations/overrides for critical actions.  
- **Access control:** RBAC and per‑agent permissions.  
- **Evals:** measure **accuracy**, **performance**, **reliability** continuously.


## 9) Architecture Quick‑Reference

### Layers
| Layer | Responsibilities |
|---|---|
| **Client/UI** | Input, visualization, streaming output, traces |
| **AgentOS** | API, auth, routing, streaming, scaling |
| **Agents/Teams/Workflows** | Reasoning, collaboration, orchestration |
| **Tools/Knowledge/MCP** | External actions and retrieval |
| **Storage** | Sessions, history, state, vector indexes |

### Performance Focus
| Dimension | What Agno optimizes |
|---|---|
| **Agent** | Instantiation time, low memory, efficient history |
| **System** | Async API, stateless scale‑out, background tasks |
| **Reliability** | Evals, schema validation, hooks/guardrails |


## 10) Minimal Runtime References (do not run without setup)

**Single agent + AgentOS**


In [ ]:
from agno.agent import Agent
from agno.models.anthropic import Claude
from agno.os import AgentOS

agent = Agent(name="Helper", model=Claude(id="claude-sonnet-4-5"), markdown=True)
agent_os = AgentOS(agents=[agent])
app = agent_os.get_app()

if __name__ == "__main__":
    agent_os.serve(app="app:app", reload=True)

**Team example (sketch)**

In [ ]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.team import Team

researcher = Agent(name="Researcher", model=OpenAIChat(id="gpt-3.5-turbo"))
analyst    = Agent(name="Analyst",    model=OpenAIChat(id="gpt-3.5-turbo"))
writer     = Agent(name="Writer",     model=OpenAIChat(id="gpt-3.5-turbo"))

qa_team = Team(members=[researcher, analyst, writer],
               instructions="Work together: research, analyze, then write a concise answer.")

**Workflow example (sketch)**

In [ ]:
# Pseudocode-style sketch for a workflow class (API varies by version)
# class MyWorkflow(Workflow):
#     step1 = parse_intent_agent
#     router = Router(...)
#     step2a = coding_agent
#     step2b = qa_agent
#     step3 = validator
# wf = MyWorkflow(); print(wf.run("User question"))